# 🍺 기네스의 비밀: Student's t-분포의 탄생

## 📖 1908년 더블린의 이야기

1908년, 아일랜드 더블린의 **세인트 제임스 게이트 양조장**에서 일하던 젊은 화학자가 있었습니다. 그의 이름은 **William Sealy Gosset**. 🧑‍🔬

매일 아침, Gosset은 고민에 빠져있었습니다:

> "맥주의 품질을 확인하려면 많은 샘플이 필요한데... 😰  
> 하루에 겨우 3-4병만 검사할 수 있어.  
> Z-검정은 σ를 알아야 하는데, 우리는 표본표준편차 s만 있고...  
> 도대체 어떻게 해야 할까?"

이 고민이 바로 **통계학 역사상 가장 중요한 발견** 중 하나로 이어지게 됩니다! 🌟

---

## 🎯 학습 목표

이 노트북에서 우리는:
1. **작은 표본 문제**의 본질을 이해합니다
2. **t-분포의 탄생 배경**을 스토리로 따라갑니다
3. **몬테카를로 시뮬레이션**으로 t-분포를 직접 유도합니다
4. **자유도의 의미**를 시각적으로 탐구합니다
5. **Z-검정 vs t-검정**의 차이를 체감합니다

In [ ]:
print("🍺 기네스 양조장으로의 시간여행 준비 완료!")
print("📅 1908년 더블린으로 떠납니다...")

## 🧪 1. Gosset의 문제: 작은 표본의 딜레마

### 🏭 기네스 양조장의 현실

Gosset이 직면한 현실적 제약들:

1. **비용 문제** 💰
   - 맥주 한 병을 검사하면 판매할 수 없음
   - 많은 샘플 = 큰 손실

2. **시간 제약** ⏰
   - 화학 분석에는 시간이 오래 걸림
   - 하루에 3-4개 샘플이 한계

3. **알려지지 않은 σ** ❓
   - 새로운 배치마다 품질이 다름
   - 과거 데이터로부터 σ를 정확히 알 수 없음

### 🤔 핵심 질문

> **σ 대신 s(표본표준편차)를 사용하면 어떻게 될까?**

$$Z = \\frac{\\bar{X} - \\mu}{\\sigma/\\sqrt{n}} \\quad \\text{vs} \\quad T = \\frac{\\bar{X} - \\mu}{s/\\sqrt{n}}$$

좌측은 **알려진 σ** (완벽한 세상), 우측은 **추정된 s** (현실 세상)

In [ ]:
# 🍺 기네스 맥주 품질 검사 시뮬레이션
def guinness_quality_simulation(n_samples=4, true_alcohol=5.0, true_std=0.3, n_days=1000):
    \"\"\"기네스 양조장의 일일 품질 검사 시뮬레이션\"\"\"
    
    np.random.seed(42)
    
    daily_means = []
    daily_stds = []
    z_statistics = []  # σ를 알 때
    t_statistics = []  # σ를 모를 때 (s 사용)
    
    print(f\"🍺 기네스 양조장 품질 검사 시뮬레이션\")
    print(f\"📊 실제 알코올 도수: {true_alcohol}% (표준편차: {true_std}%)\")
    print(f\"🔬 일일 검사 샘플 수: {n_samples}개\")
    print(f\"📅 시뮬레이션 기간: {n_days}일\")
    print(\"\")
    
    for day in tqdm(range(n_days), desc=\"품질 검사 진행 중\"):
        # 하루치 맥주 샘플 검사
        daily_samples = np.random.normal(true_alcohol, true_std, n_samples)
        
        daily_mean = np.mean(daily_samples)
        daily_std = np.std(daily_samples, ddof=1)  # 표본표준편차
        
        daily_means.append(daily_mean)
        daily_stds.append(daily_std)
        
        # Z-통계량 (σ=0.3 알려진 경우)
        z_stat = (daily_mean - true_alcohol) / (true_std / np.sqrt(n_samples))
        z_statistics.append(z_stat)
        
        # t-통계량 (s 사용하는 경우)
        if daily_std > 0:  # 0으로 나누기 방지
            t_stat = (daily_mean - true_alcohol) / (daily_std / np.sqrt(n_samples))
            t_statistics.append(t_stat)
    
    return {
        'daily_means': daily_means,
        'daily_stds': daily_stds,
        'z_statistics': z_statistics,
        't_statistics': t_statistics,
        'n_samples': n_samples,
        'true_alcohol': true_alcohol,
        'true_std': true_std
    }

def visualize_gosset_discovery(simulation_data):
    \"\"\"Gosset의 발견을 시각화\"\"\"
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            '1. 일일 알코올 도수 측정값',
            '2. 일일 표본표준편차 변동',
            '3. Z vs t 통계량 분포 비교',
            '4. 이론 vs 실제 분포'
        ],
        specs=[[
            {'type': 'scatter'}, {'type': 'histogram'}
        ], [
            {'type': 'histogram'}, {'type': 'scatter'}
        ]]
    )
    
    # 1. 일일 측정값 시계열
    days = list(range(1, len(simulation_data['daily_means']) + 1))
    fig.add_trace(
        go.Scatter(
            x=days[:100],  # 처음 100일만 표시
            y=simulation_data['daily_means'][:100],
            mode='lines+markers',
            name='일일 평균 도수',
            line=dict(color='blue')
        ),
        row=1, col=1
    )
    
    # 목표 알코올 도수 선
    fig.add_hline(
        y=simulation_data['true_alcohol'],
        line_dash=\"dash\",
        line_color=\"red\",
        annotation_text=f\"목표: {simulation_data['true_alcohol']}%\",
        row=1, col=1
    )
    
    # 2. 표본표준편차 히스토그램
    fig.add_trace(
        go.Histogram(
            x=simulation_data['daily_stds'],
            nbinsx=50,
            name='표본표준편차 s',
            marker_color='orange',
            opacity=0.7
        ),
        row=1, col=2
    )
    
    # 실제 σ 선
    fig.add_vline(
        x=simulation_data['true_std'],
        line_dash=\"dash\",
        line_color=\"red\",
        annotation_text=f\"실제 σ = {simulation_data['true_std']}\",
        row=1, col=2
    )
    
    # 3. Z vs t 통계량 비교
    fig.add_trace(
        go.Histogram(
            x=simulation_data['z_statistics'],
            nbinsx=50,
            name='Z-통계량 (σ 알려짐)',
            marker_color='blue',
            opacity=0.6,
            histnorm='probability density'
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Histogram(
            x=simulation_data['t_statistics'],
            nbinsx=50,
            name='t-통계량 (σ 모름)',
            marker_color='red',
            opacity=0.6,
            histnorm='probability density'
        ),
        row=2, col=1
    )
    
    # 4. 이론적 분포 곡선
    x_range = np.linspace(-4, 4, 200)
    
    # 표준정규분포 (Z)
    y_normal = stats.norm.pdf(x_range, 0, 1)
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=y_normal,
            mode='lines',
            name='표준정규분포 N(0,1)',
            line=dict(color='blue', width=3)
        ),
        row=2, col=2
    )
    
    # t-분포 (자유도 = n-1)
    df = simulation_data['n_samples'] - 1
    y_t = stats.t.pdf(x_range, df)
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=y_t,
            mode='lines',
            name=f't-분포 (df={df})',
            line=dict(color='red', width=3, dash='dash')
        ),
        row=2, col=2
    )
    
    # 레이아웃 업데이트
    fig.update_layout(
        title=f'🍺 Gosset의 발견: 작은 표본 (n={simulation_data[\"n_samples\"]})에서의 문제',
        height=800,
        showlegend=True
    )
    
    # 축 레이블
    fig.update_xaxes(title_text=\"일수\", row=1, col=1)
    fig.update_yaxes(title_text=\"알코올 도수 (%)\", row=1, col=1)
    fig.update_xaxes(title_text=\"표본표준편차\", row=1, col=2)
    fig.update_xaxes(title_text=\"통계량 값\", row=2, col=1)
    fig.update_xaxes(title_text=\"값\", row=2, col=2)
    fig.update_yaxes(title_text=\"확률밀도\", row=2, col=2)
    
    return fig

# 기네스 품질 검사 시뮬레이션 실행
guinness_data = guinness_quality_simulation(n_samples=4, n_days=1000)
fig = visualize_gosset_discovery(guinness_data)
fig.show()

# 통계 요약
z_stats = guinness_data['z_statistics']
t_stats = guinness_data['t_statistics']

print(f\"\"\"\n🔍 Gosset의 관찰 결과:\n
📊 Z-통계량 (σ 알려진 경우):\n
   평균: {np.mean(z_stats):.3f} (이론값: 0.000)\n
   표준편차: {np.std(z_stats):.3f} (이론값: 1.000)\n
\n
📊 t-통계량 (σ 모르는 경우):\n
   평균: {np.mean(t_stats):.3f} (이론값: 0.000)\n
   표준편차: {np.std(t_stats):.3f} (이론값: {np.sqrt(3/(4-2)):.3f} for df=3)\n
\n
🎯 핵심 발견:\n
   💡 t-통계량이 Z-통계량보다 더 넓게 퍼져있다!\n
   💡 이는 s를 사용함으로써 생기는 추가적인 불확실성 때문이다\n
   💡 표본 크기가 작을수록 이 차이가 더 크다\n
\"\"\")

## 🎭 2. "Student"의 탄생: 비밀스러운 필명

### 🤫 왜 \"Student\"였을까?

1908년, Gosset은 그의 혁신적인 발견을 논문으로 발표했습니다. 하지만 **실명 대신 "Student"라는 필명**을 사용했습니다. 왜일까요?

#### 🏭 기업의 비밀주의
- **기네스는 직원들의 학술 발표를 금지**했습니다
- 통계적 품질 관리는 **경쟁사에게 유리한 정보**였기 때문
- Gosset은 **회사 몰래** 연구를 발표해야 했습니다

#### 📚 겸손한 과학자
- Gosset은 자신을 **"학생(Student)"**이라고 생각했습니다
- 위대한 통계학자들 앞에서 **배우는 입장**이라는 겸손함
- 이 필명이 **역사상 가장 유명한 분포 이름**이 되었습니다!

### 📄 역사적 논문

> **"The Probable Error of a Mean"**  
> by Student  
> Biometrika, Vol. 6, No. 1 (Mar., 1908), pp. 1-25

이 25페이지짜리 논문이 **현대 통계학의 기초**를 놓았습니다! 📚✨

In [ ]:
# 📜 Student의 원리 재현: 몬테카를로로 t-분포 유도하기
def derive_t_distribution_monte_carlo(sample_size=5, n_simulations=10000):
    \"\"\"몬테카를로 시뮬레이션으로 t-분포 유도\"\"\"
    
    print(f\"🎲 몬테카를로 시뮬레이션으로 t-분포 유도하기\")
    print(f\"📊 표본 크기: {sample_size}\")
    print(f\"🔄 시뮬레이션 횟수: {n_simulations:,}회\")
    print(\"\")
    
    np.random.seed(42)
    
    # 표준정규분포 N(0,1)에서 표본들을 뽑는다고 가정
    # (일반성을 잃지 않음 - 모든 정규분포는 표준화 가능)
    true_mean = 0
    true_std = 1
    
    t_statistics = []
    sample_means = []
    sample_stds = []
    
    for i in tqdm(range(n_simulations), desc=\"t-분포 유도 중\"):
        # 표본 추출
        sample = np.random.normal(true_mean, true_std, sample_size)
        
        # 표본통계량 계산
        sample_mean = np.mean(sample)
        sample_std = np.std(sample, ddof=1)  # 표본표준편차
        
        # t-통계량 계산
        if sample_std > 0:  # 0으로 나누기 방지
            t_stat = (sample_mean - true_mean) / (sample_std / np.sqrt(sample_size))
            t_statistics.append(t_stat)
        
        sample_means.append(sample_mean)
        sample_stds.append(sample_std)
    
    return {
        't_statistics': t_statistics,
        'sample_means': sample_means,
        'sample_stds': sample_stds,
        'sample_size': sample_size,
        'df': sample_size - 1
    }

def visualize_t_distribution_derivation(monte_carlo_data):
    \"\"\"몬테카를로로 유도한 t-분포 시각화\"\"\"
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            '1. 표본평균들의 분포',
            '2. 표본표준편차들의 분포',
            '3. 유도된 t-분포 vs 이론적 t-분포',
            '4. t-분포 vs 표준정규분포'
        ]
    )
    
    # 1. 표본평균들의 분포
    fig.add_trace(
        go.Histogram(
            x=monte_carlo_data['sample_means'],
            nbinsx=50,
            name='표본평균들',
            marker_color='lightblue',
            opacity=0.7,
            histnorm='probability density'
        ),
        row=1, col=1
    )
    
    # 이론적 분포 (표본평균)
    x_mean = np.linspace(-2, 2, 100)
    y_mean_theory = stats.norm.pdf(x_mean, 0, 1/np.sqrt(monte_carlo_data['sample_size']))
    fig.add_trace(
        go.Scatter(
            x=x_mean,
            y=y_mean_theory,
            mode='lines',
            name='이론적 분포',
            line=dict(color='red', width=3)
        ),
        row=1, col=1
    )
    
    # 2. 표본표준편차들의 분포
    fig.add_trace(
        go.Histogram(
            x=monte_carlo_data['sample_stds'],
            nbinsx=50,
            name='표본표준편차들',
            marker_color='lightgreen',
            opacity=0.7
        ),
        row=1, col=2
    )
    
    # 실제 표준편차 선
    fig.add_vline(
        x=1.0,
        line_dash=\"dash\",
        line_color=\"red\",
        annotation_text=\"σ = 1\",
        row=1, col=2
    )
    
    # 3. 유도된 t-분포 vs 이론적 t-분포
    fig.add_trace(
        go.Histogram(
            x=monte_carlo_data['t_statistics'],
            nbinsx=50,
            name='몬테카를로 t-통계량',
            marker_color='orange',
            opacity=0.6,
            histnorm='probability density'
        ),
        row=2, col=1
    )
    
    # 이론적 t-분포
    x_t = np.linspace(-5, 5, 200)
    df = monte_carlo_data['df']
    y_t_theory = stats.t.pdf(x_t, df)
    fig.add_trace(
        go.Scatter(
            x=x_t,
            y=y_t_theory,
            mode='lines',
            name=f'이론적 t-분포 (df={df})',
            line=dict(color='red', width=3)
        ),
        row=2, col=1
    )
    
    # 4. t-분포 vs 표준정규분포 비교
    y_normal = stats.norm.pdf(x_t, 0, 1)
    
    fig.add_trace(
        go.Scatter(
            x=x_t,
            y=y_t_theory,
            mode='lines',
            name=f't-분포 (df={df})',
            line=dict(color='red', width=3)
        ),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=x_t,
            y=y_normal,
            mode='lines',
            name='표준정규분포',
            line=dict(color='blue', width=3, dash='dash')
        ),
        row=2, col=2
    )
    
    # 레이아웃
    fig.update_layout(
        title=f'📜 Student의 원리: 몬테카를로로 t-분포 유도 (n={monte_carlo_data[\"sample_size\"]}, df={df})',
        height=800,
        showlegend=True
    )
    
    return fig

# 인터랙티브 t-분포 유도
@interact(
    sample_size=IntSlider(min=3, max=20, step=1, value=5, description='표본크기 (n)'),
    n_simulations=widgets.Dropdown(
        options=[('빠름 (1,000회)', 1000), ('보통 (5,000회)', 5000), ('정확 (10,000회)', 10000)],
        value=5000,
        description='시뮬레이션:'
    )
)
def interactive_t_derivation(sample_size, n_simulations):
    # 몬테카를로 시뮬레이션
    mc_data = derive_t_distribution_monte_carlo(sample_size, n_simulations)
    
    # 시각화
    fig = visualize_t_distribution_derivation(mc_data)
    fig.show()
    
    # 통계적 검증
    t_stats = mc_data['t_statistics']
    df = mc_data['df']
    
    # 이론값과 비교
    theoretical_mean = 0
    theoretical_var = df / (df - 2) if df > 2 else np.inf
    theoretical_std = np.sqrt(theoretical_var) if df > 2 else np.inf
    
    empirical_mean = np.mean(t_stats)
    empirical_std = np.std(t_stats)
    
    print(f\"\"\"\n🎯 Student의 발견 검증:\n
    📊 자유도 (df): {df}\n
    📊 이론적 평균: {theoretical_mean:.3f}  |  실제 평균: {empirical_mean:.3f}\n
    📊 이론적 표준편차: {theoretical_std:.3f}  |  실제 표준편차: {empirical_std:.3f}\n
    📐 평균 오차: {abs(theoretical_mean - empirical_mean):.4f}\n
    📐 표준편차 오차: {abs(theoretical_std - empirical_std):.4f}\n
    \n
    💡 핵심 통찰:\n
    {'✅ 우리가 몬테카를로로 유도한 분포가 이론적 t-분포와 일치합니다!' if abs(theoretical_std - empirical_std) < 0.1 else '⚠️ 시뮬레이션 횟수를 늘려보세요!'}\n
    🎉 이것이 바로 1908년 Gosset이 발견한 것입니다!\n
    \"\"\")

## 📊 3. 자유도의 비밀: df = n - 1

### 🤔 왜 자유도는 n-1일까?

t-분포의 가장 중요한 특징 중 하나는 **자유도(degrees of freedom, df)**입니다.

#### 🧮 자유도의 직관적 이해

표본 크기가 n=3인 경우를 생각해보세요:
- 데이터: [x₁, x₂, x₃]
- 표본평균: $\\bar{x} = \\frac{x_1 + x_2 + x_3}{3}$

**핵심 질문**: 표본표준편차를 계산할 때, 몇 개의 값이 \"자유롭게\" 변할 수 있을까요?

$$s^2 = \\frac{\\sum_{i=1}^{n} (x_i - \\bar{x})^2}{n-1}$$

#### 🔒 제약 조건

편차들의 합은 항상 0입니다:
$$(x_1 - \\bar{x}) + (x_2 - \\bar{x}) + (x_3 - \\bar{x}) = 0$$

이것은 **선형 제약 조건**입니다! 
- 두 개의 편차를 알면, 세 번째는 **자동으로 결정**됩니다
- 따라서 **독립적으로 변할 수 있는 편차는 2개** (= n-1)
- 이것이 바로 **자유도**입니다!

### 📈 자유도가 t-분포에 미치는 영향

- **df가 작을수록**: 분포가 더 넓고, 꼬리가 더 두껍다
- **df가 클수록**: 표준정규분포에 가까워진다
- **df → ∞**: 완전히 표준정규분포가 된다

In [ ]:
# 📊 자유도의 영향 시각화
def visualize_degrees_of_freedom(df_list=[1, 2, 3, 5, 10, 30]):
    \"\"\"자유도가 t-분포에 미치는 영향 시각화\"\"\"
    
    x = np.linspace(-4, 4, 1000)
    
    fig = go.Figure()
    
    # 각 자유도별 t-분포
    colors = px.colors.qualitative.Set1
    
    for i, df in enumerate(df_list):
        y = stats.t.pdf(x, df)
        fig.add_trace(
            go.Scatter(
                x=x, y=y,
                mode='lines',
                name=f't-분포 (df={df})',
                line=dict(width=2.5, color=colors[i % len(colors)])
            )
        )
    
    # 표준정규분포 추가
    y_normal = stats.norm.pdf(x, 0, 1)
    fig.add_trace(
        go.Scatter(
            x=x, y=y_normal,
            mode='lines',
            name='표준정규분포 N(0,1)',
            line=dict(width=3, color='black', dash='dash')
        )
    )
    
    # 레이아웃
    fig.update_layout(
        title='🔢 자유도(df)가 t-분포에 미치는 영향',
        xaxis_title='값',
        yaxis_title='확률밀도',
        template='plotly_white',
        height=600,
        legend=dict(x=0.7, y=0.95)
    )
    
    return fig

def degrees_of_freedom_comparison(df_values=[1, 3, 10, 30]):
    \"\"\"자유도별 주요 통계량 비교\"\"\"
    
    comparison_data = []
    
    for df in df_values:
        # 주요 통계량 계산
        mean = 0  # 항상 0
        
        if df > 2:
            variance = df / (df - 2)
            std = np.sqrt(variance)
        else:
            variance = np.inf
            std = np.inf
        
        # 95% 임계값
        critical_95 = stats.t.ppf(0.975, df)
        
        # 99% 임계값
        critical_99 = stats.t.ppf(0.995, df)
        
        comparison_data.append({
            '자유도 (df)': df,
            '평균': mean,
            '분산': variance,
            '표준편차': std,
            '95% 임계값': critical_95,
            '99% 임계값': critical_99
        })
    
    # 표준정규분포 추가
    comparison_data.append({
        '자유도 (df)': '∞ (정규분포)',
        '평균': 0,
        '분산': 1,
        '표준편차': 1,
        '95% 임계값': 1.96,
        '99% 임계값': 2.576
    })
    
    df_comparison = pd.DataFrame(comparison_data)
    
    return df_comparison

# 자유도 영향 시각화
fig_df = visualize_degrees_of_freedom()
fig_df.show()

# 자유도별 비교표
df_table = degrees_of_freedom_comparison()
print(\"\"\"\n📊 자유도별 t-분포 특성 비교:\n\"\"\")
print(df_table.round(3))

print(\"\"\"\n\n💡 핵심 관찰:\n
🎯 자유도가 증가할수록:\n
   • 분산이 1에 가까워진다 (표준정규분포와 같아짐)\n
   • 임계값이 작아진다 (더 정확한 검정 가능)\n
   • 분포의 꼬리가 얇아진다\n
\n
🎯 실용적 의미:\n
   • 작은 표본(df < 10): t-분포를 반드시 사용해야 함\n
   • 큰 표본(df > 30): 정규분포 근사 가능\n
   • df = 1~2: 분산이 무한대 (매우 불안정)\n
\"\"\")

In [ ]:
# 🎲 자유도 체험: 인터랙티브 시뮬레이션
def interactive_degrees_of_freedom_demo():
    \"\"\"자유도의 의미를 체험할 수 있는 인터랙티브 데모\"\"\"
    
    @interact(
        df=IntSlider(min=1, max=30, step=1, value=5, description='자유도 (df)'),
        show_comparison=widgets.Checkbox(value=True, description='정규분포와 비교')
    )
    def plot_t_vs_normal(df, show_comparison):
        x = np.linspace(-4, 4, 1000)
        
        fig = go.Figure()
        
        # t-분포
        y_t = stats.t.pdf(x, df)
        fig.add_trace(
            go.Scatter(
                x=x, y=y_t,
                mode='lines',
                name=f't-분포 (df={df})',
                line=dict(width=3, color='red')
            )
        )
        
        if show_comparison:
            # 표준정규분포
            y_normal = stats.norm.pdf(x, 0, 1)
            fig.add_trace(
                go.Scatter(
                    x=x, y=y_normal,
                    mode='lines',
                    name='표준정규분포',
                    line=dict(width=3, color='blue', dash='dash')
                )
            )
        
        # 95% 신뢰구간 영역 표시
        alpha = 0.05
        t_critical = stats.t.ppf(1 - alpha/2, df)
        
        # 가운데 95% 영역
        x_middle = x[(x >= -t_critical) & (x <= t_critical)]
        y_middle = stats.t.pdf(x_middle, df)
        
        fig.add_trace(
            go.Scatter(
                x=np.concatenate([x_middle, x_middle[::-1]]),
                y=np.concatenate([y_middle, np.zeros(len(y_middle))]),
                fill='toself',
                fillcolor='rgba(0, 255, 0, 0.2)',
                line=dict(width=0),
                name='95% 신뢰영역',
                showlegend=True
            )
        )
        
        # 임계값 선들
        for t_val in [-t_critical, t_critical]:
            fig.add_vline(
                x=t_val,
                line_dash=\"dash\",
                line_color=\"orange\",
                annotation_text=f't = {t_val:.3f}'
            )
        
        # 비교를 위한 정규분포 임계값
        if show_comparison:
            z_critical = 1.96
            for z_val in [-z_critical, z_critical]:
                fig.add_vline(
                    x=z_val,
                    line_dash=\"dot\",
                    line_color=\"blue\",
                    annotation_text=f'Z = {z_val:.3f}'
                )
        
        # 표본크기 정보 추가
        sample_size = df + 1
        fig.update_layout(
            title=f'자유도 {df} (표본크기 n={sample_size})에서의 t-분포',
            xaxis_title='값',
            yaxis_title='확률밀도',
            template='plotly_white',
            height=500
        )
        
        fig.show()
        
        # 통계 정보 출력
        if df > 2:
            variance = df / (df - 2)
            std = np.sqrt(variance)
        else:
            variance = \"∞\"
            std = \"∞\"
        
        normal_critical = 1.96
        difference = abs(t_critical - normal_critical)
        
        print(f\"\"\"\n📊 자유도 {df} (표본크기 n={sample_size}) 분석:\n
        📈 분산: {variance}\n
        📏 표준편차: {std}\n
        🎯 95% 임계값: ±{t_critical:.3f}\n
        📊 정규분포 임계값: ±{normal_critical:.3f}\n
        📐 차이: {difference:.3f}\n
        \n
        💡 해석:\n
        {'🎯 t-분포가 정규분포보다 넓습니다 (더 보수적)' if difference > 0.1 else '✅ t-분포가 정규분포에 거의 근사합니다'}\n
        {'⚠️ 작은 표본에서는 t-분포를 반드시 사용해야 합니다!' if difference > 0.5 else ''}\n
        \"\"\")

# 인터랙티브 데모 실행
interactive_degrees_of_freedom_demo()

## ⚔️ 4. Z-검정 vs t-검정: 결정적 대결

이제 우리는 두 세계를 비교할 수 있습니다:
- **완벽한 세상** (Z-검정): σ를 알고 있음
- **현실 세상** (t-검정): σ를 모르고 s를 사용

### ⚡ 핵심 차이점

| 특성 | Z-검정 | t-검정 |
|------|--------|--------|
| **모집단 표준편차** | σ 알려짐 ✅ | σ 모름 ❌ |
| **사용하는 표준편차** | σ (정확한 값) | s (추정값) |
| **검정통계량 분포** | 표준정규분포 N(0,1) | t-분포 (df=n-1) |
| **임계값** | 고정 (예: ±1.96) | df에 따라 변함 |
| **표본크기 의존성** | 무관 | 작을수록 더 보수적 |
| **현실 적용성** | 제한적 🚨 | 매우 높음 ✨ |

### 🔄 검정통계량 공식 비교

**Z-검정**:
$$Z = \\frac{\\bar{X} - \\mu_0}{\\sigma/\\sqrt{n}} \\sim N(0,1)$$

**t-검정**:
$$t = \\frac{\\bar{X} - \\mu_0}{s/\\sqrt{n}} \\sim t_{n-1}$$

차이는 단 하나: **σ vs s**. 하지만 이 작은 차이가 **통계학 혁명**을 일으켰습니다!

In [ ]:
# ⚔️ Z-검정 vs t-검정 비교 시뮬레이션
def z_vs_t_test_comparison(true_mean=100, true_std=15, hypothesized_mean=95, 
                          sample_sizes=[5, 10, 20, 50], alpha=0.05, n_simulations=1000):
    \"\"\"Z-검정과 t-검정의 성능 비교\"\"\"
    
    np.random.seed(42)
    
    results = []
    
    for n in sample_sizes:
        z_rejections = 0
        t_rejections = 0
        z_p_values = []
        t_p_values = []
        z_statistics = []
        t_statistics = []
        
        for _ in range(n_simulations):
            # 표본 생성
            sample = np.random.normal(true_mean, true_std, n)
            sample_mean = np.mean(sample)
            sample_std = np.std(sample, ddof=1)
            
            # Z-검정 (σ 알려진 경우)
            z_stat = (sample_mean - hypothesized_mean) / (true_std / np.sqrt(n))
            z_p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))  # 양측검정
            z_reject = z_p_value < alpha
            
            # t-검정 (σ 모르는 경우)
            t_stat = (sample_mean - hypothesized_mean) / (sample_std / np.sqrt(n))
            t_p_value = 2 * (1 - stats.t.cdf(abs(t_stat), n-1))  # 양측검정
            t_reject = t_p_value < alpha
            
            # 결과 수집
            z_rejections += z_reject
            t_rejections += t_reject
            z_p_values.append(z_p_value)
            t_p_values.append(t_p_value)
            z_statistics.append(z_stat)
            t_statistics.append(t_stat)
        
        # 검정력 계산 (귀무가설이 거짓일 때 기각하는 비율)
        z_power = z_rejections / n_simulations
        t_power = t_rejections / n_simulations
        
        results.append({
            'sample_size': n,
            'z_power': z_power,
            't_power': t_power,
            'z_p_values': z_p_values,
            't_p_values': t_p_values,
            'z_statistics': z_statistics,
            't_statistics': t_statistics
        })
    
    return results

def visualize_z_vs_t_comparison(comparison_results):
    \"\"\"Z-검정 vs t-검정 비교 결과 시각화\"\"\"
    
    n_samples = len(comparison_results)
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            '1. 검정력 비교 (높을수록 좋음)',
            '2. p-값 분포 비교 (n=5)',
            '3. 통계량 분포 비교 (n=5)',
            '4. 표본크기별 임계값 차이'
        ]
    )
    
    # 1. 검정력 비교
    sample_sizes = [r['sample_size'] for r in comparison_results]
    z_powers = [r['z_power'] for r in comparison_results]
    t_powers = [r['t_power'] for r in comparison_results]
    
    fig.add_trace(
        go.Scatter(
            x=sample_sizes,
            y=z_powers,
            mode='lines+markers',
            name='Z-검정 검정력',
            line=dict(color='blue', width=3),
            marker=dict(size=8)
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=sample_sizes,
            y=t_powers,
            mode='lines+markers',
            name='t-검정 검정력',
            line=dict(color='red', width=3),
            marker=dict(size=8)
        ),
        row=1, col=1
    )
    
    # 2. p-값 분포 (n=5)
    small_sample_result = comparison_results[0]  # n=5
    
    fig.add_trace(
        go.Histogram(
            x=small_sample_result['z_p_values'],
            nbinsx=30,
            name='Z-검정 p-값',
            marker_color='blue',
            opacity=0.6
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Histogram(
            x=small_sample_result['t_p_values'],
            nbinsx=30,
            name='t-검정 p-값',
            marker_color='red',
            opacity=0.6
        ),
        row=1, col=2
    )
    
    # 유의수준 선
    fig.add_vline(
        x=0.05,
        line_dash=\"dash\",
        line_color=\"black\",
        annotation_text=\"α = 0.05\",
        row=1, col=2
    )
    
    # 3. 통계량 분포 (n=5)
    fig.add_trace(
        go.Histogram(
            x=small_sample_result['z_statistics'],
            nbinsx=40,
            name='Z-통계량',
            marker_color='blue',
            opacity=0.6,
            histnorm='probability density'
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Histogram(
            x=small_sample_result['t_statistics'],
            nbinsx=40,
            name='t-통계량',
            marker_color='red',
            opacity=0.6,
            histnorm='probability density'
        ),
        row=2, col=1
    )
    
    # 이론적 분포 곡선 추가
    x_range = np.linspace(-6, 6, 200)
    y_normal = stats.norm.pdf(x_range, 0, 1)
    y_t = stats.t.pdf(x_range, 4)  # df = n-1 = 4
    
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=y_normal,
            mode='lines',
            name='표준정규분포',
            line=dict(color='blue', width=2, dash='dash')
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=y_t,
            mode='lines',
            name='t-분포 (df=4)',
            line=dict(color='red', width=2, dash='dash')
        ),
        row=2, col=1
    )
    
    # 4. 임계값 차이
    z_critical = 1.96  # 95% 양측검정
    t_criticals = [stats.t.ppf(0.975, n-1) for n in sample_sizes]
    differences = [t_crit - z_critical for t_crit in t_criticals]
    
    fig.add_trace(
        go.Scatter(
            x=sample_sizes,
            y=[z_critical] * len(sample_sizes),
            mode='lines',
            name='Z 임계값 (1.96)',
            line=dict(color='blue', width=3, dash='dash')
        ),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=sample_sizes,
            y=t_criticals,
            mode='lines+markers',
            name='t 임계값',
            line=dict(color='red', width=3),
            marker=dict(size=8)
        ),
        row=2, col=2
    )
    
    # 레이아웃
    fig.update_layout(
        title='⚔️ Z-검정 vs t-검정: 완벽한 세상 vs 현실 세상',
        height=800,
        showlegend=True
    )
    
    # 축 레이블
    fig.update_xaxes(title_text=\"표본크기\", row=1, col=1)
    fig.update_yaxes(title_text=\"검정력\", row=1, col=1)
    fig.update_xaxes(title_text=\"p-값\", row=1, col=2)
    fig.update_xaxes(title_text=\"통계량 값\", row=2, col=1)
    fig.update_yaxes(title_text=\"확률밀도\", row=2, col=1)
    fig.update_xaxes(title_text=\"표본크기\", row=2, col=2)
    fig.update_yaxes(title_text=\"임계값\", row=2, col=2)
    
    return fig, differences

# 비교 시뮬레이션 실행
print(\"🎯 Z-검정 vs t-검정 성능 비교 시뮬레이션 시작...\")
print(\"📊 설정: 실제 평균=100, 가설 평균=95, σ=15\")
print(\"\")

comparison_results = z_vs_t_test_comparison(
    true_mean=100, true_std=15, hypothesized_mean=95,
    sample_sizes=[5, 10, 20, 50], n_simulations=1000
)

fig_comparison, critical_differences = visualize_z_vs_t_comparison(comparison_results)
fig_comparison.show()

# 결과 요약
print(\"\"\"\n🏆 승부 결과 분석:\n\"\"\")

for i, result in enumerate(comparison_results):
    n = result['sample_size']
    z_power = result['z_power']
    t_power = result['t_power']
    diff = critical_differences[i]
    
    print(f\"\"\"\n📊 표본크기 n = {n}:\n
    🎯 Z-검정 검정력: {z_power:.3f}\n
    🎯 t-검정 검정력: {t_power:.3f}\n
    📐 임계값 차이: {diff:.3f} (t가 더 큼)\n
    {'✅ Z-검정 승리!' if z_power > t_power else '✅ t-검정 승리!' if t_power > z_power else '🤝 무승부!'}\"\"\")

print(\"\"\"\n\n💡 핵심 통찰:\n
🎯 검정력 관점: Z-검정이 약간 우세 (σ를 정확히 알기 때문)\n
🎯 현실성 관점: t-검정이 압승 (σ를 모르는 것이 현실)\n
🎯 작은 표본: t-검정의 보수적 접근이 더 안전\n
🎯 큰 표본: 두 검정의 차이가 거의 없어짐\n
\n
🏆 결론: 현실에서는 t-검정이 왕이다! 👑\"\"\")

## 🎉 5. Gosset의 유산: 1908년의 혁명

### 🌍 통계학계에 미친 영향

William Gosset의 발견은 단순한 수학적 호기심이 아니었습니다. **실용적 필요**에서 나온 혁신이었죠!

#### 🚀 즉각적인 영향
- **농업 연구**: 작은 구획에서의 작물 실험
- **의학 연구**: 제한된 환자 수로도 약물 효과 검증
- **품질 관리**: 비용 효율적인 제품 검사
- **사회과학**: 설문조사와 심리학 실험

#### 📈 장기적인 유산
- **현대 임상시험의 기초**
- **A/B 테스트의 통계적 근거**
- **머신러닝에서의 모델 검증**
- **데이터 사이언스의 필수 도구**

### 🎭 "Student"에서 "Gosset"으로

1922년, 기네스가 정책을 바꾸면서 Gosset은 비로소 실명으로 활동할 수 있게 되었습니다. 하지만 이미 **"Student's t-distribution"**이라는 이름은 영원히 역사에 새겨졌습니다.

### 💭 Gosset의 철학

> "이론을 위한 이론이 아니라, **실제 문제 해결을 위한 통계학**이어야 한다"  
> - William Sealy Gosset

이 철학이 바로 **현대 응용통계학의 정신**입니다!

In [ ]:
# 🎉 Gosset의 유산: 현대적 응용 사례
def modern_applications_of_t_test():
    \"\"\"t-검정의 현대적 응용 사례들을 시뮬레이션\"\"\"
    
    # 1. 임상시험: 신약 효과 검증
    print(\"🏥 사례 1: 신약 임상시험\")
    print(\"상황: 새로운 혈압약의 효과를 12명의 환자로 테스트\")
    
    np.random.seed(42)
    # 치료 전후 혈압 차이 (실제로는 5mmHg 감소 효과)
    blood_pressure_reduction = np.random.normal(5, 3, 12)
    
    # 일표본 t-검정: 평균 감소량이 0보다 큰가?
    t_stat, p_value = stats.ttest_1samp(blood_pressure_reduction, 0)
    
    print(f\"\"\"\n📊 결과:\n
    평균 혈압 감소: {np.mean(blood_pressure_reduction):.2f} mmHg\n
    t-통계량: {t_stat:.3f}\n
    p-값: {p_value:.4f}\n
    결론: {'신약에 효과가 있습니다!' if p_value < 0.05 else '효과가 명확하지 않습니다.'}\n
    \"\"\")
    
    # 2. A/B 테스트: 웹사이트 전환율
    print(\"\n💻 사례 2: 웹사이트 A/B 테스트\")
    print(\"상황: 새로운 버튼 디자인이 클릭률을 높이는가?\")
    
    # 기존 디자인 vs 새 디자인 (각각 20명씩)
    old_design_clicks = np.random.binomial(1, 0.12, 20)  # 12% 클릭률
    new_design_clicks = np.random.binomial(1, 0.18, 20)  # 18% 클릭률
    
    # 독립 t-검정
    t_stat_ab, p_value_ab = stats.ttest_ind(new_design_clicks, old_design_clicks)
    
    print(f\"\"\"\n📊 결과:\n
    기존 디자인 클릭률: {np.mean(old_design_clicks):.1%}\n
    새 디자인 클릭률: {np.mean(new_design_clicks):.1%}\n
    t-통계량: {t_stat_ab:.3f}\n
    p-값: {p_value_ab:.4f}\n
    결론: {'새 디자인이 더 효과적입니다!' if p_value_ab < 0.05 else '차이가 명확하지 않습니다.'}\n
    \"\"\")
    
    # 3. 교육 연구: 학습 방법 비교
    print(\"\n📚 사례 3: 교육 방법 효과 연구\")
    print(\"상황: 새로운 교육 방법이 학생들의 성적을 향상시키는가?\")
    
    # 동일 학생들의 교육 전후 점수 (대응표본)
    before_scores = np.random.normal(75, 10, 15)
    improvement = np.random.normal(8, 5, 15)  # 평균 8점 향상
    after_scores = before_scores + improvement
    
    # 대응표본 t-검정
    t_stat_paired, p_value_paired = stats.ttest_rel(after_scores, before_scores)
    
    print(f\"\"\"\n📊 결과:\n
    교육 전 평균: {np.mean(before_scores):.1f}점\n
    교육 후 평균: {np.mean(after_scores):.1f}점\n
    평균 향상: {np.mean(improvement):.1f}점\n
    t-통계량: {t_stat_paired:.3f}\n
    p-값: {p_value_paired:.4f}\n
    결론: {'새 교육 방법이 효과적입니다!' if p_value_paired < 0.05 else '효과가 명확하지 않습니다.'}\n
    \"\"\")
    
    return {
        'clinical': (t_stat, p_value),
        'ab_test': (t_stat_ab, p_value_ab),
        'education': (t_stat_paired, p_value_paired)
    }

# 현대적 응용 사례 실행
modern_results = modern_applications_of_t_test()

print(\"\"\"\n\n🌟 Gosset의 1908년 발견이 오늘날 어떻게 활용되는지 보셨나요?\n
🎯 핵심 포인트:\n
   • 작은 표본으로도 과학적 결론 도출 가능\n
   • 비용 효율적인 실험 설계\n
   • 불확실성을 정량화하여 신뢰할 수 있는 의사결정\n
\n
🍺 기네스 맥주 한 잔의 품질 고민이 현대 과학의 기초가 되었습니다!\"\"\")

## 📚 핵심 개념 요약

### ✨ 오늘의 여행에서 배운 것들

1. **작은 표본의 문제** 🔬
   - σ를 모르고 s를 사용할 때의 추가적 불확실성
   - 비용과 시간 제약으로 인한 현실적 한계

2. **Student's t-분포의 탄생** 🍺
   - William Gosset의 실용적 필요에서 출발
   - 몬테카를로 시뮬레이션으로 직접 유도 가능

3. **자유도의 의미** 🔢
   - df = n-1 (독립적으로 변할 수 있는 편차의 개수)
   - 자유도가 분포의 형태를 결정

4. **Z-검정 vs t-검정** ⚔️
   - 완벽한 세상 vs 현실 세상
   - t-검정이 더 보수적이지만 현실적

5. **현대적 응용** 🚀
   - 임상시험, A/B 테스트, 교육 연구 등
   - 작은 표본으로도 과학적 결론 도출

### 🔑 핵심 공식들

- **t-검정 통계량**: $t = \\frac{\\bar{X} - \\mu_0}{s/\\sqrt{n}} \\sim t_{n-1}$
- **자유도**: $df = n - 1$
- **t-분포 분산**: $\\text{Var}(t) = \\frac{df}{df-2}$ (df > 2일 때)

---

## 🧩 연습 문제

### 문제 1: 자유도 이해
표본크기가 8일 때, t-분포의 자유도는 얼마이고, 95% 양측검정의 임계값은?

### 문제 2: Gosset의 상황 재현
맥주 4병의 알코올 도수가 [4.8, 5.2, 4.9, 5.1]%였습니다. 목표 도수 5.0%와 차이가 있는지 t-검정으로 확인하세요.

### 문제 3: Z vs t 비교
동일한 데이터로 Z-검정(σ=0.2 가정)과 t-검정을 수행했을 때 어떤 차이가 있을까요?

---

## 🚀 다음 여행지: "t-test 삼총사"

이제 우리는 t-분포의 비밀을 알았습니다! 🎉

다음 노트북에서는 **세 가지 다른 상황**에서 사용하는 t-검정들을 만나보겠습니다:

- **One-sample t-test**: "우리 제품이 기준을 만족하는가?"
- **Independent t-test**: "A와 B 그룹이 정말 다른가?"
- **Paired t-test**: "치료 전후 차이가 있는가?"

각각 언제, 어떻게 사용하는지 실제 사례와 함께 탐구해보겠습니다!

**다음 노트북**: `03_types_of_t_tests.ipynb`